In [1]:
import pandas as pd
import ibm_db

In [2]:
import os, types
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_be27c39a06954f219332d6d4c9ebfebb = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='odBnNMkPuimz6CZzOOda9OQwb_dhnAzB30CDSvzoHxOD',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

body = client_be27c39a06954f219332d6d4c9ebfebb.get_object(Bucket='db2testproject-donotdelete-pr-n2jaljjbpb4fa7',Key='name_age_weight.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

people_df = pd.read_csv(body)
people_df.head()


,Name,Age,Weight
0,Mike,56,100.8
1,Robert,34,210.9
2,Amber,45,105.6
3,Anna,23,204.8


In [7]:
# Enter the values for you database connection. 
dsn_database = ""            # e.g. "MORTGAGE"
dsn_uid      = ""            # e.g. "dash104434"
dsn_pwd      = ""            # e.g. "7dBZ3jWt9xN6$o0JiX!m"
dsn_hostname = ""            # e.g. "Use the same IP as Web Console"
dsn_port     = ""               # e.g. "50000" 
dsn_protocol = "TCPIP"               # i.e. "TCPIP"
dsn_driver   = "IBM DB2 ODBC DRIVER" # Don't change

In [8]:
dsn = ("DRIVER={{IBM DB2 ODBC DRIVER}};" "DATABASE={0};" "HOSTNAME={1};" "PORT={2};" "PROTOCOL=TCPIP;" "UID={3};" "PWD={4};SECURITY=SSL").format(dsn_database, dsn_hostname, dsn_port, dsn_uid, dsn_pwd)
options = { ibm_db.SQL_ATTR_AUTOCOMMIT:  ibm_db.SQL_AUTOCOMMIT_ON }
conn = ibm_db.connect(dsn, "", "",options)

In [9]:
columns = 'Person_Name,Person_Age,Person_Weight'
insertSQL = 'Insert into NAME_AGE_WEIGHT(' + columns + ') values(?,?,?)'
stmt = ibm_db.prepare(conn, insertSQL)
insertSQL

'Insert into NAME_AGE_WEIGHT(Person_Name,Person_Age,Person_Weight) values(?,?,?)'

In [12]:
for n in range(len(people_df)) :
    ibm_db.bind_param(stmt,1,str(people_df.at[n,'Name']), ibm_db.SQL_PARAM_INPUT,ibm_db.SQL_VARCHAR)
    ibm_db.bind_param(stmt,2,str(people_df.at[n,'Age']), ibm_db.SQL_PARAM_INPUT,ibm_db.SQL_INTEGER)
    ibm_db.bind_param(stmt,3,str(people_df.at[n,'Weight']), ibm_db.SQL_PARAM_INPUT,ibm_db.SQL_DECIMAL)
    
    ibm_db.execute(stmt)

In [6]:
ibm_db.close(conn)

True